<a href="https://colab.research.google.com/github/Chau-Nguyen-Developer/HeHeHaHa_DataRoyale/blob/chau/Chau2_RoyaleClash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyzing Winning Decks

In [1]:
!pip install gdown

In [2]:
import gdown

In [3]:
file_id = "1sgMvwiuZNyBt86JD3FRVZa5YRi6DpbBj"

In [4]:
url = f"https://drive.google.com/uc?id={file_id}"

In [5]:
# Download the file
gdown.download(url, "dataset.csv", quiet=False)
print("\nFinished downloading dataset.")

Downloading...
From (original): https://drive.google.com/uc?id=1sgMvwiuZNyBt86JD3FRVZa5YRi6DpbBj
From (redirected): https://drive.google.com/uc?id=1sgMvwiuZNyBt86JD3FRVZa5YRi6DpbBj&confirm=t&uuid=ddd3a20f-f30f-4493-a9fd-082a118abe49
To: /content/dataset.csv
100%|██████████| 9.87G/9.87G [02:13<00:00, 74.2MB/s]


Finished downloading dataset.


Will only load 800,000 entries out of ~1M entries due to limited RAM


In [7]:
import pandas as pd
data = pd.read_csv("dataset.csv", nrows=800_000)
print("Done loading.")

Done loading.


In [9]:
data.columns

Index(['Unnamed: 0', 'battleTime', 'arena.id', 'gameMode.id',
       'average.startingTrophies', 'winner.tag', 'winner.startingTrophies',
       'winner.trophyChange', 'winner.crowns', 'winner.kingTowerHitPoints',
       'winner.princessTowersHitPoints', 'winner.clan.tag',
       'winner.clan.badgeId', 'loser.tag', 'loser.startingTrophies',
       'loser.trophyChange', 'loser.crowns', 'loser.kingTowerHitPoints',
       'loser.clan.tag', 'loser.clan.badgeId', 'loser.princessTowersHitPoints',
       'tournamentTag', 'winner.card1.id', 'winner.card1.level',
       'winner.card2.id', 'winner.card2.level', 'winner.card3.id',
       'winner.card3.level', 'winner.card4.id', 'winner.card4.level',
       'winner.card5.id', 'winner.card5.level', 'winner.card6.id',
       'winner.card6.level', 'winner.card7.id', 'winner.card7.level',
       'winner.card8.id', 'winner.card8.level', 'winner.cards.list',
       'winner.totalcard.level', 'winner.troop.count',
       'winner.structure.count', 'winner.

In [10]:
data["winner.common.count"]

,winner.common.count
0,1
1,4
2,1
3,2
4,3
...,...
799995,4
799996,3
799997,4
799998,1


### 1. Extract out only winner players

In [11]:
import pandas as pd

winnerCols = ['battleTime',
       'winner.tag',
       'winner.startingTrophies',
       'winner.trophyChange',
       'winner.crowns',
       'winner.cards.list',
       'winner.troop.count',
       'winner.structure.count',
       'winner.spell.count',
       'winner.common.count',
       'winner.rare.count',
       'winner.epic.count',
       'winner.legendary.count',
       'winner.elixir.average']

Make a deep copy of data[winnerCols] subset

In [12]:
winnersDataFrame = data[winnerCols].copy()

In [13]:
winnersDataFrame.head()

,battleTime,winner.tag,winner.startingTrophies,winner.trophyChange,winner.crowns,winner.cards.list,winner.troop.count,winner.structure.count,winner.spell.count,winner.common.count,winner.rare.count,winner.epic.count,winner.legendary.count,winner.elixir.average
0,2020-12-07 07:00:00+00:00,#28RR8PJP0,6581.0,31.0,2.0,"[26000036, 26000043, 26000044, 26000050, 26000...",7,0,1,1,2,3,2,3.625
1,2020-12-07 07:00:00+00:00,#YV9VQUVP,5592.0,28.0,3.0,"[26000012, 26000017, 26000022, 26000024, 26000...",5,0,3,4,2,1,1,4.125
2,2020-12-07 07:00:02+00:00,#LPR2G0Q9L,5678.0,31.0,3.0,"[26000004, 26000036, 26000042, 26000046, 26000...",6,0,2,1,1,2,4,3.875
3,2020-12-07 07:00:03+00:00,#2GL899VCJ,6035.0,29.0,2.0,"[26000004, 26000006, 26000007, 26000011, 26000...",6,0,2,2,1,5,0,3.875
4,2020-12-07 07:00:06+00:00,#9Y2YJPGG2,5140.0,30.0,3.0,"[26000004, 26000011, 26000013, 26000022, 26000...",5,0,3,3,2,2,1,4.125


Convert `winner.cards.list` (text format) into a Python list so that we can later loop over and do more work on it.

In [14]:
# Import the Abstract Syntax Trees library. It can safely evaluate strings that contain Python literals.
import ast

Evaluates the string literal into a real Python object.

In [15]:
def parseDeck(deckStr):
  try:
    return ast.literal_eval(deckStr) if isinstance(deckStr, str) else deckStr
    # Above: Check whether the value is a string and evaluates the string literal into a python object
  except:
    return []

In [16]:
winnersDataFrame["decksID"] = winnersDataFrame["winner.cards.list"].apply(parseDeck)

In [17]:
winnersDataFrame["decksID"][0]

[26000036,
 26000043,
 26000044,
 26000050,
 26000054,
 26000062,
 28000015,
 28000016]

In [18]:
type(winnersDataFrame["decksID"][0])

list

### Count card frequencies in the winning decks
Count how often each card ID appears

In [20]:
from collections import Counter

In [24]:
cardCounter = Counter()
# Counter is a special dictionary from the collections module. It counts how many times each item appears.

# Loop through row by row and count how many "winning decks" each card appeared in
for deck in winnersDataFrame["decksID"].dropna():
  #For this one deck, please increment the count of each card ID in my counter
  cardCounter.update(deck)

### See the top 20 most frequently used cards


In [30]:
print(cardCounter.most_common(20))

[(28000008, 235217), (28000011, 224663), (28000000, 203814), (26000011, 199895), (26000017, 181079), (26000012, 169697), (28000001, 165761), (26000021, 157995), (26000000, 149029), (26000015, 143995), (26000055, 142229), (26000042, 134811), (26000049, 129032), (28000004, 121542), (26000018, 120590), (26000006, 115311), (26000041, 111072), (26000014, 96689), (26000037, 96504), (28000015, 91389)]


### See the top 10 cards that appear most often in winning decks

In [31]:
print(cardCounter.most_common(10))

[(28000008, 235217), (28000011, 224663), (28000000, 203814), (26000011, 199895), (26000017, 181079), (26000012, 169697), (28000001, 165761), (26000021, 157995), (26000000, 149029), (26000015, 143995)]


In [35]:
len(cardCounter)

102

^^ Above: there are 102 unique cards.

### Calculate the percentage of each card

In [40]:
cardUsage = pd.DataFrame(cardCounter.items(), columns=["cardID", "winCount"])
cardUsage["usagePercentage"] = round((cardUsage["winCount"] / (len(winnersDataFrame))) * 100,2)
cardUsage.sort_values("winCount", ascending=False, inplace=True)
cardUsage.head(25)

,cardID,winCount,usagePercentage
13,28000008,235217,29.40
14,28000011,224663,28.08
12,28000000,203814,25.48
21,26000011,199895,24.99
9,26000017,181079,22.63
8,26000012,169697,21.21
23,28000001,165761,20.72
38,26000021,157995,19.75
34,26000000,149029,18.63
29,26000015,143995,18.00


^^ Above: the top 25 cards that showed up the most in winning decks

### Find the most common deck combinations


In [41]:
winnersDataFrame["deckTuple"] = winnersDataFrame["decksID"].apply(lambda x: tuple(sorted(x)))

^^ Above: sort the list into a consistent order. Why do we have to sort? -> Sorting helps us count decks with the identical cards but in a different order as "the same deck" later.

We also convert each list into a tuple (an immutable data type). --> will use the tuple as a key when counting duplicate tuples.

In [42]:
deckCounts = winnersDataFrame["deckTuple"].value_counts().reset_index()

.value_counts() --> we will count how many times each unique tuple appears in the column.

In [43]:
type(deckCounts)

pandas.core.frame.DataFrame

In [44]:
deckCounts

,deckTuple,count
0,"(26000000, 26000026, 26000030, 26000041, 27000...",11479
1,"(26000010, 26000014, 26000021, 26000030, 26000...",8788
2,"(26000004, 26000036, 26000042, 26000046, 26000...",7984
3,"(26000000, 26000001, 26000010, 26000030, 27000...",6564
4,"(26000006, 26000008, 26000029, 26000032, 26000...",4112
...,...,...
175343,"(26000007, 26000009, 26000018, 26000048, 26000...",1
175344,"(26000007, 26000012, 26000013, 26000015, 26000...",1
175345,"(26000000, 26000004, 26000026, 26000041, 27000...",1
175346,"(26000017, 26000021, 26000027, 26000035, 26000...",1


### Look at the top 10 most popular decks among the winners

In [45]:
deckCounts.head(10)

,deckTuple,count
0,"(26000000, 26000026, 26000030, 26000041, 27000...",11479
1,"(26000010, 26000014, 26000021, 26000030, 26000...",8788
2,"(26000004, 26000036, 26000042, 26000046, 26000...",7984
3,"(26000000, 26000001, 26000010, 26000030, 27000...",6564
4,"(26000006, 26000008, 26000029, 26000032, 26000...",4112
5,"(26000000, 26000010, 26000023, 27000006, 27000...",3625
6,"(26000009, 26000015, 26000035, 26000039, 26000...",3162
7,"(26000000, 26000019, 26000032, 26000049, 26000...",3134
8,"(26000000, 26000014, 26000019, 26000041, 26000...",3017
9,"(26000009, 26000015, 26000035, 26000039, 26000...",2993


### A little bit check in:
^^ Above: We now have a list of the most common winning deck combinations in our dataset.